In [71]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import xgboost
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from tqdm.auto import tqdm

# Call sys.path.append only once
# import os, sys; 
# sys.path.append(os.path.dirname(os.path.realpath('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/Neuro-symbolic-AI/column.py')))

# sys.path.remove(os.path.dirname(os.path.realpath('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/Neuro-symbolic-AI/column.py')))
import column


tqdm.pandas()

In [91]:
print(sys.path)

['c:\\Users\\sgopalakrish\\Downloads\\intellizenz-model-training\\baseline\\Features', 'C:\\Users\\sgopalakrish\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip', 'C:\\Users\\sgopalakrish\\AppData\\Local\\Programs\\Python\\Python39\\DLLs', 'C:\\Users\\sgopalakrish\\AppData\\Local\\Programs\\Python\\Python39\\lib', 'C:\\Users\\sgopalakrish\\AppData\\Local\\Programs\\Python\\Python39', 'c:\\Users\\sgopalakrish\\AppData\\Local\\pypoetry\\Cache\\virtualenvs\\intellizenz-model-training-LRwJb8pv-py3.9', '', 'c:\\Users\\sgopalakrish\\AppData\\Local\\pypoetry\\Cache\\virtualenvs\\intellizenz-model-training-LRwJb8pv-py3.9\\lib\\site-packages', 'c:\\Users\\sgopalakrish\\AppData\\Local\\pypoetry\\Cache\\virtualenvs\\intellizenz-model-training-LRwJb8pv-py3.9\\lib\\site-packages\\win32', 'c:\\Users\\sgopalakrish\\AppData\\Local\\pypoetry\\Cache\\virtualenvs\\intellizenz-model-training-LRwJb8pv-py3.9\\lib\\site-packages\\win32\\lib', 'c:\\Users\\sgopalakrish\\AppData\\Local\\pypoetry\\Cache

In [3]:
# df = pd.read_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_features_2016_2020_v2.parquet.gzip')
df = pd.read_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_features_2016_2020_v3.parquet.gzip')

In [4]:
pd.set_option('display.max_columns', None)

In [10]:
df=df.rename(columns = {'band_x':'band', 'vg_state_x':'vg_state'})

In [6]:
# Split the data set and extract time series features
X = df.loc[:,~df.columns.isin(['veranst_segment','vg_inkasso'])]
y = df['vg_inkasso']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [5]:
display(df.head(5))

,imp_id,gj,import,mufo_referenz_n,barcode_nr,veranst_segment,rekla_jn,vg_datum_von,vg_ort,vg_raum,nutzfall,nutzfall_raum,musikleiter_name,kapelle_name,tarif_nr,nutzfall_nr,vg_inkasso,inkasso_netto,inkasso_brutto,veranst_geschaeftszeichen,veranst_name,veranst_strasse,veranst_plz,veranst_ort,nutzliznehm_geschaeftszeichen,nutzliznehm_name,nutzliznehm_vorname,nutzliznehm_strasse,nutzliznehm_ort,location,vg_raum_clean,vg_raum_wo_stopwords,vg_raum_keywords,place_kirche,place_hotel,place_cafe,place_theater,place_club,place_halle,place_gaststaette,place_festhalle,place_kulturzentrum,place_festzelt,place_schloss,place_pub,place_stadthalle,place_park,place_gasthof,place_kabarett,place_arena,place_schlachthof,place_wandelhalle,place_turnhalle,place_buergerhaus,place_museum,place_rathaus,place_staatsbad,place_zelt,place_jazz,place_forum,place_gymnasium,place_schule,place_sporthalle,tarif_bez,tarif_u-v ii. 1 (+ii 2),tarif_u-k (musiker),tarif_u-k (musiker) mindestverguetung,tarif_u-st i. (musiker),tarif_u-k i (+ ii 2b) - mit sonstigem geldwerten vorteil,tarif_u-st i (musiker) nl,tarif_u-v iii. 1,tarif_u-v iii. 2,tarif_u-k iii. 2d) (musiker) - vor geladenen gaesten,tarif_variete i (musiker),tarif_u-v vi. b,tarif_e (musiker),tarif_p-k i. (u-musik),tarif_u-k ii. (musiker) bis 50 min,tarif_u-k ii. (musiker) bis 20 min,tarif_e-p,tarif_u-k ii. (musiker) bis 25 min,tarif_u-k ii. (musiker) bis 15 min,tarif_u-k ii. (musiker) bis 30 min,tarif_u-k ii. (musiker) bis 10 min,tarif_u-st (musiker),tarif_chorverband konzert u-musik,tarif_vk i 3 zirkusunternehmen (musiker),tarif_u-k ii. (musiker) bis 5 min,tarif_u-k ii. (musiker) bis 35 min,tarif_u-k ii. (musiker) bis 40 min,tarif_p-k i. (e-musik),tarif_u-k ii (musiker) mindestverguetung,tarif_u-k ii. (musiker) bis 45 min,tarif_u-v vi. b mindestverguetung,vg_state,state_bavaria,state_thuringia,state_baden-wuerttemberg,state_rhineland-palatinate,state_brandenburg,state_berlin,state_lower saxony,state_schleswig-holstein,state_hamburg,state_hesse,state_saarland,state_north rhine-westphalia,state_saxony-anhalt,state_saxony,state_mecklenburg-western pomerania,state_bremen,band,band_kurorchester bad wil,band_musikverein harmonie,band_kasalla,band_cat ballou,band_roncalli royal orch,band_jugendblasorchester,band_kurorchester bad pyr,band_hoehner,band_paveier,band_domstuermer,band_kluengelkoepp,band_alleinunterhalter,band_the gregorian voices,band_brings,band_musica hungarica,band_concerto,band_bad salzuflen orches,band_musikverein stadtkap,band_salonorchester hunga,band_miljoe,band_raeuber,band_kabarett leipziger f,band_marita koellner,band_salon-orchester hung,band_blaeck foeoess,band_schuelerinnen und sc,band_romain vicente,band_staatliche kurkapell,band_musikzug der freiwil,band_funky marys,promoter,promoter_clean,promoter_live nation gmbh,promoter_fkp scorpio konzertproduktionen gmbh,promoter_trinity music gmbh,promoter_karsten jahnke konzertdirektion gmbh,promoter_prime entertainment gmbh,promoter_semmel concerts entertainment gmbh,promoter_chorverband nrw e.v,promoter_schwaebischer chorverband e.v,promoter_backstage concerts gmbh,promoter_kulturzentrum schlachthof wiesbaden e.v,promoter_irish pubs gaststaetten gmbh,promoter_fraenkischer saengerbund e.v,promoter_frankfurter kulturzentrum e.v,promoter_circus roncalli gmbh,promoter_paul daly und paul fleming gbr,promoter_foerderkreis jazz und malerei muenchen e.v,promoter_hessischer saengerbund e.v,promoter_gastro event gmbh,promoter_berninger musik & gastronomie gmbh,promoter_feierwerk e.v,promoter_bayerisches staatsbad bad steben gmbh,promoter_europa-park gmbh & co mack kg,promoter_badischer chorverband 1862 e.v,promoter_bayerisches wirtshaus berlin gmbh,promoter_tollwood gmbh,promoter_graeflicher park gmbh & co. kg,promoter_konzertbuero schoneberg gmbh,promoter_staatsbad salzuflen gmbh,promoter_kurverwaltung bad mergentheim gmbh,promoter_gisbert hiller,vg_datum_year,vg_datum_month,vg_datum_day_of_week,vg_datum_season
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [24]:
print(len(df.band.unique()))#317011 - unique bands
print(len(df.promoter.unique()))#176456 - unique promoter/organizer
print(len(df.vg_state.unique()))#16 - unique bundesland/states
print(len(df.tarif_bez.unique()))#87 - unique Tarif

317007
176455
16
87


In [9]:
print(len(X_train))

1393757


# Extract inkasso distribution with respect to the time of the event

In [36]:
def extract_features(features, target):
    stat_var = ['band']
    # stat_var = ['vg_state']
    # stat_var = ['promoter']
    # stat_var = ['tarif_bez']
    df_stat = {}

    # 1. Take vg_inkasso value before the current datetime - (split the train and test and calculate the inkasso distribution-after feature calculation)
    # 2. Timestamp as index, and have location/band as columns in df
    # 3. Predefine the single df and iterate over it

    def get_descr_stat(row):
        # row represents each line(row) in the dataframe
        def descr_stat(row, var):
            # Initialize a empty pandas dataframe
            result = pd.Series(dtype='float64')

            if not pd.isnull(row[var]):
                #get vg_inkasso collected for past events(that occured before current date of the row) for each location/state, each band and each promoter
                matching_indexes = features[(features['vg_datum_von']<row['vg_datum_von'])&(features[var] == row[var])].index.values.tolist()

                inkasso =  list(target[target.index.isin(matching_indexes)].values)

                # Returns the mean, count, std, max, min and 0.05, 0.10..0.95 percentiles of the list of vg_inkasso values
                # for a specific attribute like location, band, promotor
                result = pd.Series(inkasso, dtype='float64').describe(percentiles=percentiles)
                result = result.add_prefix('{}_'.format(var))

            result.name = row.name
            return result

        descr_stat_result = pd.Series(dtype='float64')
        for v in stat_var:
            descr_stat_result = pd.concat([descr_stat_result, descr_stat(row, v)])

        return descr_stat_result

 
    
    percentiles = [round(x, 2) for x in np.linspace(0, 1, 21)[1:-1].tolist()]

    # Each row of the dataframe 'df' is passed as row argument for get_descr_stat function
    df_featurize_descr_stat = features.progress_apply(get_descr_stat, axis=1)
    # df_featurize_descr_stat.to_pickle('./data/export_descr_stat_2016_2020.pkl.bz2', protocol=4)

    # display(df_featurize_descr_stat.head(5))
    return df_featurize_descr_stat

# For training data - split the data and extract features
# For testing data - Features can be extracted without splits

# 1393757
# Splitting 1393792 entries to 128 splits, where each split contains 10889 entries
X_train_splits = np.array_split(X_train,200)
y_train_splits = np.array_split(y_train,200)

# X_train_splits = np.array_split(X_test,200)
# y_train_splits = np.array_split(y_test,200)

df_featurize_descr_stat_result = pd.Series(dtype='float64')
for i in range(0,len(X_train_splits)):
    X_train_slice = X_train_splits[i]
    y_train_slice = y_train_splits[i]
    result = extract_features(X_train_slice, y_train_slice)
    df_featurize_descr_stat_result = pd.concat([df_featurize_descr_stat_result, result])

df_featurize_descr_stat_result.index.name = 'ID'
df_featurize_descr_stat_result = df_featurize_descr_stat_result.drop(0, axis=1) # remove 0th index column with NaN values 
display(df_featurize_descr_stat_result.head(5))

# df_featurize_descr_stat = extract_features(X_test, y_test)
# display(df_featurize_descr_stat.head(5))

c:\Users\sgopalakrish\AppData\Local\pypoetry\Cache\virtualenvs\intellizenz-model-training-LRwJb8pv-py3.9\lib\site-packages\numpy\lib\shape_base.py:790: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  sub_arys.append(_nx.swapaxes(sary[st:end], axis, 0))


  0%|          | 0/1743 [00:00<?, ?it/s]

  0%|          | 0/1743 [00:00<?, ?it/s]

  0%|          | 0/1743 [00:00<?, ?it/s]

  0%|          | 0/1743 [00:00<?, ?it/s]

...

  0%|          | 0/1742 [00:00<?, ?it/s]

  0%|          | 0/1742 [00:00<?, ?it/s]

  0%|          | 0/1742 [00:00<?, ?it/s]

  0%|          | 0/1742 [00:00<?, ?it/s]

,band_count,band_mean,band_std,band_min,band_5%,band_10%,band_15%,band_20%,band_25%,band_30%,band_35%,band_40%,band_45%,band_50%,band_55%,band_60%,band_65%,band_70%,band_75%,band_80%,band_85%,band_90%,band_95%,band_max
ID,,,,,,,,,,,,,,,,,,,,,,,,
10363475,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11910688,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12389286,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10580453,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11270659,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Train set

In [20]:
# Training set - Saving band feature inkasso distribution
# df_featurize_descr_stat_result.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')
df_featurize_descr_stat_result.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_train_band_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip',compression='gzip')

In [16]:
# Training set - Saving vg_state feature inkasso distribution
# df_featurize_descr_stat_result.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_vg_state_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')
df_featurize_descr_stat_result.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_train_vg_state_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip',compression='gzip')

In [23]:
# Training set - Saving promoter feature inkasso distribution
# df_featurize_descr_stat_result.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_promoter_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')
df_featurize_descr_stat_result.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_train_promoter_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip',compression='gzip')

In [28]:
# Training set - Saving Tarif feature inkasso distribution
df_featurize_descr_stat_result.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_train_tarif_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip',compression='gzip')

### Test set

In [31]:
# Testing set - Saving promoter feature inkasso distribution
# df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_promoter_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')
df_featurize_descr_stat_result.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_test_promoter_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip',compression='gzip')

In [37]:
# Testing set - Saving band feature inkasso distribution
# df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')
df_featurize_descr_stat_result.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_test_band_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip',compression='gzip')

In [33]:
# Testing set - Saving vg_state feature inkasso distribution
# df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_vg_state_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')
df_featurize_descr_stat_result.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_test_vg_state_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip',compression='gzip')

In [35]:
# Testing set - Saving Tarif feature inkasso distribution
df_featurize_descr_stat_result.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_test_tarif_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip',compression='gzip')

## Merge the extracted features into training set and testing set

In [39]:
# Merge the time series distribution of vg_inkasso values for training data of band, promoter, vg_state, tarif
# data_path_train_vg_state = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_vg_state_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
# data_path_train_band = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
# data_path_train_promoter = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_train_promoter_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')

data_path_train_vg_state = Path('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_train_vg_state_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip')
data_path_train_band = Path('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_train_band_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip')
data_path_train_promoter = Path('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_train_promoter_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip')
data_path_train_tarif = Path('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_train_tarif_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip')

if data_path_train_vg_state.exists() and data_path_train_band.exists() and data_path_train_promoter.exists() and data_path_train_tarif.exists():
    df_train_state = pd.read_parquet(data_path_train_vg_state)
    df_train_band = pd.read_parquet(data_path_train_band)
    df_train_promoter = pd.read_parquet(data_path_train_promoter)
    df_train_tarif = pd.read_parquet(data_path_train_tarif)

    df_train_time_series_merged_descr_stat = df_train_state.merge(df_train_band, how='left', on='ID').merge(df_train_promoter, how='left', on='ID').merge(df_train_tarif, 
                                                                                                                                                           how ='left', 
                                                                                                                                                           on='ID')

display(df_train_time_series_merged_descr_stat.head(5))
print(df_train_time_series_merged_descr_stat.shape)

,vg_state_count,vg_state_mean,vg_state_std,vg_state_min,vg_state_5%,vg_state_10%,vg_state_15%,vg_state_20%,vg_state_25%,vg_state_30%,vg_state_35%,vg_state_40%,vg_state_45%,vg_state_50%,vg_state_55%,vg_state_60%,vg_state_65%,vg_state_70%,vg_state_75%,vg_state_80%,vg_state_85%,vg_state_90%,vg_state_95%,vg_state_max,band_count,band_mean,band_std,band_min,band_5%,band_10%,band_15%,band_20%,band_25%,band_30%,band_35%,band_40%,band_45%,band_50%,band_55%,band_60%,band_65%,band_70%,band_75%,band_80%,band_85%,band_90%,band_95%,band_max,promoter_count,promoter_mean,promoter_std,promoter_min,promoter_5%,promoter_10%,promoter_15%,promoter_20%,promoter_25%,promoter_30%,promoter_35%,promoter_40%,promoter_45%,promoter_50%,promoter_55%,promoter_60%,promoter_65%,promoter_70%,promoter_75%,promoter_80%,promoter_85%,promoter_90%,promoter_95%,promoter_max,tarif_bez_count,tarif_bez_mean,tarif_bez_std,tarif_bez_min,tarif_bez_5%,tarif_bez_10%,tarif_bez_15%,tarif_bez_20%,tarif_bez_25%,tarif_bez_30%,tarif_bez_35%,tarif_bez_40%,tarif_bez_45%,tarif_bez_50%,tarif_bez_55%,tarif_bez_60%,tarif_bez_65%,tarif_bez_70%,tarif_bez_75%,tarif_bez_80%,tarif_bez_85%,tarif_bez_90%,tarif_bez_95%,tarif_bez_max
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10616298,90.0,477.414111,1454.331817,15.30,17.44,18.04,19.486,21.80,22.800,33.416,45.1000,45.600,50.2930,79.770,91.1500,98.252,124.0465,158.771,180.010,200.828,235.661,477.372,2861.0635,9244.59,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,97.42,NaN,97.42,97.42,97.42,97.42,97.42,97.42,97.42,97.42,97.42,97.42,97.42,97.42,97.42,97.42,97.42,97.42,97.42,97.42,97.42,97.42,97.42,244.0,1510.646270,15765.619873,17.44,17.44,18.240,21.690,21.80,22.80,24.527,32.49,37.184,43.3575,46.94,58.00,68.086,77.0325,100.336,130.25,185.350,278.931,394.806,1008.0325,240794.10
12954841,1561.0,716.528757,13161.711136,11.02,18.24,18.64,21.800,23.30,31.350,37.280,43.6100,50.350,59.4300,69.150,82.4000,94.200,120.5000,141.300,193.460,242.320,315.760,452.030,913.3600,510768.02,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1837.0,1329.383092,13111.160133,13.69,18.44,18.640,19.776,23.05,23.30,26.824,34.80,40.806,49.3720,61.91,77.88,100.000,125.5020,166.088,226.13,312.424,456.464,810.942,2201.6880,328956.82
12179380,731.0,261.783051,1292.127303,11.02,18.24,18.82,22.800,25.00,34.800,36.880,45.6000,54.720,61.2100,69.150,86.7400,94.000,116.1150,136.800,182.400,235.260,292.335,384.200,653.3100,28854.30,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.0,30.422129,38.522723,14.91,15.59,15.590,15.770,15.77,15.77,16.768,17.44,18.240,18.4400,18.44,21.80,22.800,23.0500,23.050,23.05,28.542,45.600,46.100,69.1500,313.48
11468891,8.0,45.488750,53.311211,17.44,17.65,17.86,18.070,18.28,18.490,18.693,18.8785,19.064,19.5645,20.485,21.4055,24.656,29.6540,34.652,42.185,50.732,59.279,94.022,133.1310,172.24,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,351.100000,NaN,351.10,351.10,351.100,351.100,351.10,351.10,351.100,351.10,351.100,351.1000,351.10,351.10,351.100,351.1000,351.100,351.10,351.100,351.100,351.100,351.1000,351.10
11528958,540.0,257.562907,1445.211470,11.02,18.24,18.44,22.800,23.05,32.185,36.480,45.0650,46.100,56.2440,67.650,73.3200,91.200,111.3785,132.742,159.700,216.096,274.035,344.424,517.1985,28854.30,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,745.0,199.781987,409.412352,12

(1393757, 96)


In [40]:
# Merge the time series distribution of vg_inkasso values for testing data of band, promoter, vg_state, tarif
# data_path_test_vg_state = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_vg_state_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
# data_path_test_band = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
# data_path_test_promoter = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_test_promoter_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')

data_path_test_vg_state = Path('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_test_vg_state_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip')
data_path_test_band = Path('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_test_band_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip')
data_path_test_promoter = Path('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_test_promoter_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip')
data_path_test_tarif = Path('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_test_tarif_stats_for_range_of_vginkasso_2016_2020_v1.parquet.gzip')

if data_path_test_vg_state.exists() and data_path_test_band.exists() and data_path_test_promoter.exists() and data_path_test_tarif.exists():
    df_test_state = pd.read_parquet(data_path_test_vg_state)
    df_test_band = pd.read_parquet(data_path_test_band)
    df_test_promoter = pd.read_parquet(data_path_test_promoter)
    df_test_tarif = pd.read_parquet(data_path_test_tarif)

    df_test_time_series_merged_descr_stat = df_test_state.merge(df_test_band, how='left', on='ID').merge(df_test_promoter,
                                                                                                            how='left',
                                                                                                            on='ID').merge(df_test_tarif, how='left', on='ID')

display(df_test_time_series_merged_descr_stat.head(5))
print(df_test_time_series_merged_descr_stat.shape)

,vg_state_count,vg_state_mean,vg_state_std,vg_state_min,vg_state_5%,vg_state_10%,vg_state_15%,vg_state_20%,vg_state_25%,vg_state_30%,vg_state_35%,vg_state_40%,vg_state_45%,vg_state_50%,vg_state_55%,vg_state_60%,vg_state_65%,vg_state_70%,vg_state_75%,vg_state_80%,vg_state_85%,vg_state_90%,vg_state_95%,vg_state_max,band_count,band_mean,band_std,band_min,band_5%,band_10%,band_15%,band_20%,band_25%,band_30%,band_35%,band_40%,band_45%,band_50%,band_55%,band_60%,band_65%,band_70%,band_75%,band_80%,band_85%,band_90%,band_95%,band_max,promoter_count,promoter_mean,promoter_std,promoter_min,promoter_5%,promoter_10%,promoter_15%,promoter_20%,promoter_25%,promoter_30%,promoter_35%,promoter_40%,promoter_45%,promoter_50%,promoter_55%,promoter_60%,promoter_65%,promoter_70%,promoter_75%,promoter_80%,promoter_85%,promoter_90%,promoter_95%,promoter_max,tarif_bez_count,tarif_bez_mean,tarif_bez_std,tarif_bez_min,tarif_bez_5%,tarif_bez_10%,tarif_bez_15%,tarif_bez_20%,tarif_bez_25%,tarif_bez_30%,tarif_bez_35%,tarif_bez_40%,tarif_bez_45%,tarif_bez_50%,tarif_bez_55%,tarif_bez_60%,tarif_bez_65%,tarif_bez_70%,tarif_bez_75%,tarif_bez_80%,tarif_bez_85%,tarif_bez_90%,tarif_bez_95%,tarif_bez_max
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10363475,7.0,65.771429,67.482009,17.44,17.6800,17.920,18.1600,18.24,18.2400,18.240,20.1660,25.944,31.722,37.500,39.9240,42.348,44.7720,64.664,93.2900,121.916,145.140,157.560,169.980,182.40,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,33.112,8.203726,18.44,22.048,25.656,29.264,32.872,36.48,36.56,36.64,36.72,36.8,36.88,36.88,36.88,36.88,36.88,36.88,36.88,36.88,36.88,36.88,36.88,171.0,169.916316,335.086806,15.59,18.24,22.550,23.050,32.830,36.480,45.100,45.8500,60.930,72.160,85.09,91.200,109.440,119.955,134.120,157.710,209.680,247.8000,315.540,519.6900,2796.21
11910688,49.0,384.054694,1933.079412,16.00,17.8400,19.704,22.0500,23.20,23.5500,27.708,33.4380,36.256,46.700,56.580,62.5260,72.020,93.2100,121.532,171.3600,187.530,221.948,274.370,489.744,13609.66,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,16.000,NaN,16.00,16.000,16.000,16.000,16.000,16.00,16.00,16.00,16.00,16.0,16.00,16.00,16.00,16.00,16.00,16.00,16.00,16.00,16.00,16.00,16.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12389286,218.0,3217.277248,40545.104954,15.59,17.3680,18.440,18.8400,23.05,24.4125,36.492,37.6800,45.600,55.710,68.505,77.3485,91.040,122.5175,151.212,209.5350,275.976,338.504,618.909,1753.025,598240.46,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414.0,1662.758213,16925.717486,15.77,18.44,18.777,20.542,22.800,23.550,31.574,35.9145,40.630,47.034,55.97,67.140,79.478,105.305,130.958,202.400,301.424,382.3530,668.343,1470.3770,326167.54
10580453,6.0,107.275000,68.596625,30.85,34.4125,37.975,41.5375,45.10,62.0125,78.925,95.8375,112.750,112.750,112.750,112.7500,112.750,114.0925,115.435,116.7775,118.120,144.610,171.100,197.590,224.08,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,107.457692,107.323821,18.04,22.55,22.550,22.550,33.988,40.745,45.100,47.8060,62.114,67.897,72.16,78.082,85.458,102.198,112.750,115.435,134.592,206.2350,225.500,370.2320,460.24
11270659,130.0,5141.152462,52491.187682,15.59,16.5215,18.240,18.9860,22.80,23.0500,31.994,36.7270,45.600,58.408,69.175,79.5000,91.600,125.1740,159.600,208.4350,255.408,328.800,448.292,1115.237,598240.46,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280.0,179.29410

(348440, 96)


In [92]:
# features = column.features_v3
# v3 features
features = ['place_kirche', 'place_hotel', 'place_cafe', 'place_theater', 'place_club', 'place_halle', 'place_gaststaette', 
'place_festhalle', 'place_kulturzentrum', 'place_festzelt', 'place_schloss', 'place_pub', 'place_stadthalle', 'place_park', 
'place_gasthof', 'place_kabarett', 'place_arena', 'place_schlachthof', 'place_wandelhalle', 'place_turnhalle', 'place_buergerhaus', 'place_museum', 
'place_rathaus', 'place_staatsbad', 'place_zelt', 'place_jazz', 'place_forum', 'place_gymnasium', 'place_schule', 'place_sporthalle', 

'tarif_u-v ii. 1 (+ii 2)', 'tarif_u-k (musiker)', 'tarif_u-k (musiker) mindestverguetung', 'tarif_u-k i (+ ii 2b) - mit sonstigem geldwerten vorteil', 
'tarif_u-st i (musiker) nl', 'tarif_u-v iii. 1', 'tarif_u-v iii. 2', 'tarif_u-k iii. 2d) (musiker) - vor geladenen gaesten', 'tarif_variete i (musiker)', 
'tarif_u-v vi. b', 'tarif_e (musiker)', 'tarif_p-k i. (u-musik)', 'tarif_u-k ii. (musiker) bis 50 min', 'tarif_u-k ii. (musiker) bis 20 min', 'tarif_e-p', 
'tarif_u-k ii. (musiker) bis 15 min', 'tarif_u-k ii. (musiker) bis 25 min', 'tarif_u-k ii. (musiker) bis 30 min', 'tarif_u-st (musiker)', 
'tarif_u-k ii. (musiker) bis 10 min', 'tarif_chorverband konzert u-musik', 'tarif_vk i 3 zirkusunternehmen (musiker)', 'tarif_u-k ii. (musiker) bis 5 min', 
'tarif_u-k ii. (musiker) bis 35 min', 'tarif_u-k ii. (musiker) bis 40 min', 'tarif_p-k i. (e-musik)', 'tarif_u-st i. (musiker)', 'tarif_u-k ii (musiker) mindestverguetung', 
'tarif_u-k ii. (musiker) bis 45 min', 'tarif_u-v vi. b mindestverguetung', 

'band_kurorchester bad wil', 'band_musikverein harmonie', 'band_kasalla', 'band_cat ballou', 'band_roncalli  royal orch', 
'band_jugendblasorchester', 'band_kurorchester bad pyr', 'band_hoehner', 'band_paveier', 'band_domstuermer', 
'band_kluengelkoepp', 'band_alleinunterhalter', 'band_the gregorian voices', 'band_brings', 'band_musica hungarica', 
'band_concerto', 'band_bad salzuflen orches', 'band_musikverein stadtkap', 'band_salonorchester hunga', 'band_miljoe', 
'band_raeuber', 'band_kabarett leipziger f', 'band_marita koellner', 'band_salon-orchester hung', 'band_blaeck foeoess', 
'band_schuelerinnen und sc', 'band_romain vicente', 'band_staatliche kurkapell', 'band_musikzug der freiwil', 'band_funky marys', 

'state_bavaria', 'state_rhineland-palatinate', 'state_baden-wuerttemberg', 'state_north rhine-westphalia', 'state_thuringia', 
'state_hesse', 'state_brandenburg', 'state_schleswig-holstein', 'state_berlin', 'state_mecklenburg-western pomerania', 
'state_lower saxony', 'state_hamburg', 'state_saarland', 'state_saxony-anhalt', 'state_saxony', 'state_bremen', 

'vg_datum_year', 'vg_datum_month', 'vg_datum_day_of_week', 'vg_datum_season', 

'promoter_live nation gmbh', 'promoter_fkp scorpio konzertproduktionen gmbh', 'promoter_trinity music gmbh', 
'promoter_karsten jahnke konzertdirektion gmbh', 'promoter_prime entertainment gmbh', 'promoter_semmel concerts entertainment gmbh', 
'promoter_chorverband nrw e.v', 'promoter_schwaebischer chorverband e.v', 'promoter_backstage concerts gmbh', 
'promoter_kulturzentrum schlachthof wiesbaden e.v', 'promoter_irish pubs gaststaetten gmbh', 'promoter_fraenkischer saengerbund e.v', 
'promoter_frankfurter kulturzentrum e.v', 'promoter_circus roncalli gmbh', 'promoter_paul daly und paul fleming gbr', 
'promoter_foerderkreis jazz und malerei muenchen e.v', 'promoter_hessischer saengerbund e.v', 'promoter_gastro event gmbh', 
'promoter_berninger musik & gastronomie gmbh', 'promoter_feierwerk e.v', 'promoter_bayerisches staatsbad bad steben gmbh', 
'promoter_europa-park gmbh & co mack kg', 'promoter_badischer chorverband 1862 e.v', 'promoter_bayerisches wirtshaus berlin gmbh', 
'promoter_tollwood gmbh', 'promoter_graeflicher park gmbh & co. kg', 'promoter_konzertbuero schoneberg gmbh', 'promoter_staatsbad salzuflen gmbh', 
'promoter_kurverwaltung bad mergentheim gmbh', 'promoter_gisbert hiller', 

'veranst_segment','vg_inkasso'
]

In [13]:
# features = [
# 'place_kirche', 'place_hotel', 'place_cafe',
#  'place_theater', 'place_club', 'place_halle',
#  'place_gaststaette', 'place_festhalle', 'place_kulturzentrum',
#  'place_festzelt', 'place_schloss', 'place_pub',
#  'place_stadthalle', 'place_park', 'place_gasthof',
#  'place_kabarett', 'place_arena', 'place_schlachthof',
#  'place_wandelhalle', 'place_turnhalle', 'place_buergerhaus',
#  'place_museum', 'place_rathaus', 'place_staatsbad',
#  'place_zelt', 'place_jazz', 'place_forum',
#  'place_gymnasium', 'place_schule', 'place_sporthalle', 

# 'band_kurorchester bad wil', 'band_musikverein harmonie', 'band_kasalla',
#  'band_cat ballou', 'band_roncalli  royal orch', 'band_jugendblasorchester',
#  'band_kurorchester bad pyr', 'band_hoehner', 'band_paveier',
#  'band_domstuermer', 'band_kluengelkoepp', 'band_alleinunterhalter',
#  'band_the gregorian voices', 'band_brings', 'band_musica hungarica',
#  'band_concerto', 'band_bad salzuflen orches', 'band_musikverein stadtkap',
#  'band_salonorchester hunga', 'band_miljoe', 'band_raeuber',
#  'band_kabarett leipziger f', 'band_marita koellner', 'band_salon-orchester hung',
#  'band_blaeck foeoess', 'band_schuelerinnen und sc', 'band_romain vicente',
#  'band_staatliche kurkapell', 'band_musikzug der freiwil', 'band_funky marys',

# 'state_bavaria','state_rhineland-palatinate',
# 'state_baden-wuerttemberg',	'state_north rhine-westphalia',	
# 'state_thuringia','state_hesse',	
# 'state_brandenburg', 'state_schleswig-holstein',	
# 'state_berlin',	'state_mecklenburg-western pomerania',	
# 'state_lower saxony', 'state_hamburg',	
# 'state_saarland', 'state_saxony-anhalt',	
# 'state_saxony',	'state_bremen',

# 'vg_datum_year','vg_datum_month','vg_datum_day_of_week','vg_datum_season', 

# 'veranst_segment','vg_inkasso'
# ]

In [94]:
df_features = df[features]

In [95]:
# Train size = 1393757
# Test size = 348440
df_training = df_train_time_series_merged_descr_stat.merge(df_features, how='left', on='ID')
print(df_training.shape)
display(df_training.head(5))

(1393757, 238)


,vg_state_count,vg_state_mean,vg_state_std,vg_state_min,vg_state_5%,vg_state_10%,vg_state_15%,vg_state_20%,vg_state_25%,vg_state_30%,vg_state_35%,vg_state_40%,vg_state_45%,vg_state_50%,vg_state_55%,vg_state_60%,vg_state_65%,vg_state_70%,vg_state_75%,vg_state_80%,vg_state_85%,vg_state_90%,vg_state_95%,vg_state_max,band_count,band_mean,band_std,band_min,band_5%,band_10%,band_15%,band_20%,band_25%,band_30%,band_35%,band_40%,band_45%,band_50%,band_55%,band_60%,band_65%,band_70%,band_75%,band_80%,band_85%,band_90%,band_95%,band_max,promoter_count,promoter_mean,promoter_std,promoter_min,promoter_5%,promoter_10%,promoter_15%,promoter_20%,promoter_25%,promoter_30%,promoter_35%,promoter_40%,promoter_45%,promoter_50%,promoter_55%,promoter_60%,promoter_65%,promoter_70%,promoter_75%,promoter_80%,promoter_85%,promoter_90%,promoter_95%,promoter_max,tarif_bez_count,tarif_bez_mean,tarif_bez_std,tarif_bez_min,tarif_bez_5%,tarif_bez_10%,tarif_bez_15%,tarif_bez_20%,tarif_bez_25%,tarif_bez_30%,tarif_bez_35%,tarif_bez_40%,tarif_bez_45%,tarif_bez_50%,tarif_bez_55%,tarif_bez_60%,tarif_bez_65%,tarif_bez_70%,tarif_bez_75%,tarif_bez_80%,tarif_bez_85%,tarif_bez_90%,tarif_bez_95%,tarif_bez_max,place_kirche,place_hotel,place_cafe,place_theater,place_club,place_halle,place_gaststaette,place_festhalle,place_kulturzentrum,place_festzelt,place_schloss,place_pub,place_stadthalle,place_park,place_gasthof,place_kabarett,place_arena,place_schlachthof,place_wandelhalle,place_turnhalle,place_buergerhaus,place_museum,place_rathaus,place_staatsbad,place_zelt,place_jazz,place_forum,place_gymnasium,place_schule,place_sporthalle,tarif_u-v ii. 1 (+ii 2),tarif_u-k (musiker),tarif_u-k (musiker) mindestverguetung,tarif_u-k i (+ ii 2b) - mit sonstigem geldwerten vorteil,tarif_u-st i (musiker) nl,tarif_u-v iii. 1,tarif_u-v iii. 2,tarif_u-k iii. 2d) (musiker) - vor geladenen gaesten,tarif_variete i (musiker),tarif_u-v vi. b,tarif_e (musiker),tarif_p-k i. (u-musik),tarif_u-k ii. (musiker) bis 50 min,tarif_u-k ii. (musiker) bis 20 min,tarif_e-p,tarif_u-k ii. (musiker) bis 15 min,tarif_u-k ii. (musiker) bis 25 min,tarif_u-k ii. (musiker) bis 30 min,tarif_u-st (musiker),tarif_u-k ii. (musiker) bis 10 min,tarif_chorverband konzert u-musik,tarif_vk i 3 zirkusunternehmen (musiker),tarif_u-k ii. (musiker) bis 5 min,tarif_u-k ii. (musiker) bis 35 min,tarif_u-k ii. (musiker) bis 40 min,tarif_p-k i. (e-musik),tarif_u-st i. (musiker),tarif_u-k ii (musiker) mindestverguetung,tarif_u-k ii. (musiker) bis 45 min,tarif_u-v vi. b mindestverguetung,band_kurorchester bad wil,band_musikverein harmonie,band_kasalla,band_cat ballou,band_roncalli royal orch,band_jugendblasorchester,band_kurorchester bad pyr,band_hoehner,band_paveier,band_domstuermer,band_kluengelkoepp,band_alleinunterhalter,band_the gregorian voices,band_brings,band_musica hungarica,band_concerto,band_bad salzuflen orches,band_musikverein stadtkap,band_salonorchester hunga,band_miljoe,band_raeuber,band_kabarett leipziger f,band_marita koellner,band_salon-orchester hung,band_blaeck foeoess,band_schuelerinnen und sc,band_romain vicente,band_staatliche kurkapell,band_musikzug der freiwil,band_funky marys,state_bavaria,state_rhineland-palatinate,state_baden-wuerttemberg,state_north rhine-westphalia,state_thuringia,state_hesse,state_brandenburg,state_schleswig-holstein,state_berlin,state_mecklenburg-western pomerania,state_lower saxony,state_hamburg,state_saarland,state_saxony-anhalt,state_saxony,state_bremen,vg_datum_year,vg_datum_month,vg_datum_day_of_week,vg_datum_season,promoter_live nation gmbh,promoter_fkp scorpio konzertproduktionen gmbh,promoter_trinity music gmbh,promoter_karsten jahnke konzertdirektion gmbh,promoter_prime entertainment gmbh,promoter_semmel concerts entertainment gmbh,promoter_chorverband nrw e.v,promoter_schwaebischer chorverband e.v,promoter_backstage concerts gmbh,promoter_kulturzentrum schlachthof wiesbaden e.v,promoter_irish pubs gaststaetten gmbh,promoter_fraenkischer saengerbund e.v,promoter_frankfurter ku

In [96]:
df_testing = df_test_time_series_merged_descr_stat.merge(df_features, how='left', on='ID')
print(df_testing.shape)
display(df_testing.head(5))

(348440, 238)


,vg_state_count,vg_state_mean,vg_state_std,vg_state_min,vg_state_5%,vg_state_10%,vg_state_15%,vg_state_20%,vg_state_25%,vg_state_30%,vg_state_35%,vg_state_40%,vg_state_45%,vg_state_50%,vg_state_55%,vg_state_60%,vg_state_65%,vg_state_70%,vg_state_75%,vg_state_80%,vg_state_85%,vg_state_90%,vg_state_95%,vg_state_max,band_count,band_mean,band_std,band_min,band_5%,band_10%,band_15%,band_20%,band_25%,band_30%,band_35%,band_40%,band_45%,band_50%,band_55%,band_60%,band_65%,band_70%,band_75%,band_80%,band_85%,band_90%,band_95%,band_max,promoter_count,promoter_mean,promoter_std,promoter_min,promoter_5%,promoter_10%,promoter_15%,promoter_20%,promoter_25%,promoter_30%,promoter_35%,promoter_40%,promoter_45%,promoter_50%,promoter_55%,promoter_60%,promoter_65%,promoter_70%,promoter_75%,promoter_80%,promoter_85%,promoter_90%,promoter_95%,promoter_max,tarif_bez_count,tarif_bez_mean,tarif_bez_std,tarif_bez_min,tarif_bez_5%,tarif_bez_10%,tarif_bez_15%,tarif_bez_20%,tarif_bez_25%,tarif_bez_30%,tarif_bez_35%,tarif_bez_40%,tarif_bez_45%,tarif_bez_50%,tarif_bez_55%,tarif_bez_60%,tarif_bez_65%,tarif_bez_70%,tarif_bez_75%,tarif_bez_80%,tarif_bez_85%,tarif_bez_90%,tarif_bez_95%,tarif_bez_max,place_kirche,place_hotel,place_cafe,place_theater,place_club,place_halle,place_gaststaette,place_festhalle,place_kulturzentrum,place_festzelt,place_schloss,place_pub,place_stadthalle,place_park,place_gasthof,place_kabarett,place_arena,place_schlachthof,place_wandelhalle,place_turnhalle,place_buergerhaus,place_museum,place_rathaus,place_staatsbad,place_zelt,place_jazz,place_forum,place_gymnasium,place_schule,place_sporthalle,tarif_u-v ii. 1 (+ii 2),tarif_u-k (musiker),tarif_u-k (musiker) mindestverguetung,tarif_u-k i (+ ii 2b) - mit sonstigem geldwerten vorteil,tarif_u-st i (musiker) nl,tarif_u-v iii. 1,tarif_u-v iii. 2,tarif_u-k iii. 2d) (musiker) - vor geladenen gaesten,tarif_variete i (musiker),tarif_u-v vi. b,tarif_e (musiker),tarif_p-k i. (u-musik),tarif_u-k ii. (musiker) bis 50 min,tarif_u-k ii. (musiker) bis 20 min,tarif_e-p,tarif_u-k ii. (musiker) bis 15 min,tarif_u-k ii. (musiker) bis 25 min,tarif_u-k ii. (musiker) bis 30 min,tarif_u-st (musiker),tarif_u-k ii. (musiker) bis 10 min,tarif_chorverband konzert u-musik,tarif_vk i 3 zirkusunternehmen (musiker),tarif_u-k ii. (musiker) bis 5 min,tarif_u-k ii. (musiker) bis 35 min,tarif_u-k ii. (musiker) bis 40 min,tarif_p-k i. (e-musik),tarif_u-st i. (musiker),tarif_u-k ii (musiker) mindestverguetung,tarif_u-k ii. (musiker) bis 45 min,tarif_u-v vi. b mindestverguetung,band_kurorchester bad wil,band_musikverein harmonie,band_kasalla,band_cat ballou,band_roncalli royal orch,band_jugendblasorchester,band_kurorchester bad pyr,band_hoehner,band_paveier,band_domstuermer,band_kluengelkoepp,band_alleinunterhalter,band_the gregorian voices,band_brings,band_musica hungarica,band_concerto,band_bad salzuflen orches,band_musikverein stadtkap,band_salonorchester hunga,band_miljoe,band_raeuber,band_kabarett leipziger f,band_marita koellner,band_salon-orchester hung,band_blaeck foeoess,band_schuelerinnen und sc,band_romain vicente,band_staatliche kurkapell,band_musikzug der freiwil,band_funky marys,state_bavaria,state_rhineland-palatinate,state_baden-wuerttemberg,state_north rhine-westphalia,state_thuringia,state_hesse,state_brandenburg,state_schleswig-holstein,state_berlin,state_mecklenburg-western pomerania,state_lower saxony,state_hamburg,state_saarland,state_saxony-anhalt,state_saxony,state_bremen,vg_datum_year,vg_datum_month,vg_datum_day_of_week,vg_datum_season,promoter_live nation gmbh,promoter_fkp scorpio konzertproduktionen gmbh,promoter_trinity music gmbh,promoter_karsten jahnke konzertdirektion gmbh,promoter_prime entertainment gmbh,promoter_semmel concerts entertainment gmbh,promoter_chorverband nrw e.v,promoter_schwaebischer chorverband e.v,promoter_backstage concerts gmbh,promoter_kulturzentrum schlachthof wiesbaden e.v,promoter_irish pubs gaststaetten gmbh,promoter_fraenkischer saengerbund e.v,promoter_frankfurter ku

## Allow only maximum of 2 NaN per row

In [38]:
test_copy = df_testing.copy()

In [39]:
# Allow only maximum of 2 NaN per row or remove the rows with more than 2 NaN values Eg: band_std = NaN and promoter_std = NaN
test_copy.dropna(axis=0, thresh=152, inplace=True)
display(test_copy.head(5))
#(151287, 154)

,vg_state_count,vg_state_mean,vg_state_std,vg_state_min,vg_state_5%,vg_state_10%,vg_state_15%,vg_state_20%,vg_state_25%,vg_state_30%,vg_state_35%,vg_state_40%,vg_state_45%,vg_state_50%,vg_state_55%,vg_state_60%,vg_state_65%,vg_state_70%,vg_state_75%,vg_state_80%,vg_state_85%,vg_state_90%,vg_state_95%,vg_state_max,band_count,band_mean,band_std,band_min,band_5%,band_10%,band_15%,band_20%,band_25%,band_30%,band_35%,band_40%,band_45%,band_50%,band_55%,band_60%,band_65%,band_70%,band_75%,band_80%,band_85%,band_90%,band_95%,band_max,promoter_10%,promoter_15%,promoter_20%,promoter_25%,promoter_30%,promoter_35%,promoter_40%,promoter_45%,promoter_5%,promoter_50%,promoter_55%,promoter_60%,promoter_65%,promoter_70%,promoter_75%,promoter_80%,promoter_85%,promoter_90%,promoter_95%,promoter_count,promoter_max,promoter_mean,promoter_min,promoter_std,place_kirche,place_hotel,place_cafe,place_theater,place_club,place_halle,place_gaststaette,place_festhalle,place_kulturzentrum,place_festzelt,place_schloss,place_pub,place_stadthalle,place_park,place_gasthof,place_kabarett,place_arena,place_schlachthof,place_wandelhalle,place_turnhalle,place_buergerhaus,place_museum,place_rathaus,place_staatsbad,place_zelt,place_jazz,place_forum,place_gymnasium,place_schule,place_sporthalle,band_kurorchester bad wil,band_musikverein harmonie,band_kasalla,band_cat ballou,band_roncalli royal orch,band_jugendblasorchester,band_kurorchester bad pyr,band_hoehner,band_paveier,band_domstuermer,band_kluengelkoepp,band_alleinunterhalter,band_the gregorian voices,band_brings,band_musica hungarica,band_concerto,band_bad salzuflen orches,band_musikverein stadtkap,band_salonorchester hunga,band_miljoe,band_raeuber,band_kabarett leipziger f,band_marita koellner,band_salon-orchester hung,band_blaeck foeoess,band_schuelerinnen und sc,band_romain vicente,band_staatliche kurkapell,band_musikzug der freiwil,band_funky marys,state_bavaria,state_rhineland-palatinate,state_baden-wuerttemberg,state_north rhine-westphalia,state_thuringia,state_hesse,state_brandenburg,state_schleswig-holstein,state_berlin,state_mecklenburg-western pomerania,state_lower saxony,state_hamburg,state_saarland,state_saxony-anhalt,state_saxony,state_bremen,vg_datum_year,vg_datum_month,vg_datum_day_of_week,vg_datum_season,veranst_segment,vg_inkasso
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10619045,8343.0,421.600133,3431.393212,10.80,17.44,18.24,22.55,22.80,34.51,36.48,45.600,54.120,65.1990,70.80,84.56,91.200,114.000,135.300,162.36,205.200,255.360,354.120,588.8300,70573.56,1.0,36.0800,NaN,36.08,36.0800,36.080,36.080,36.080,36.080,36.080,36.080,36.080,36.080,36.08,36.080,36.080,36.080,36.080,36.0800,36.080,36.0800,36.080,36.0800,36.08,18.240,18.240,18.240,18.24,18.240,18.240,18.240,18.240,18.2400,18.24,18.240,18.240,18.240,18.240,18.24,18.240,18.240,18.240,18.240,1.0,18.24,18.240000,18.24,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,9,3,4,2,18.24
11723390,41930.0,1208.057565,19775.388186,0.04,17.44,18.44,18.84,23.05,23.55,36.48,38.400,46.600,55.9200,68.47,79.54,95.526,120.000,149.120,193.60,248.570,331.920,497.880,1116.2200,689747.70,8.0,3693.8025,7388.111688,51.85,53.4285,55.007,58.775,75.680,92.585,107.946,119.447,130.948,138.141,139.59,141.039,141.660,141.660,141.660,2209.6375,5104.806,7999.9745,12040.441,16271.7905,20503.14,20503.140,20503.140,20503.140,20503.14,20503.140,20503.140,20503.140,20503.140,13430.2945,20503.14,20503.140,20503.140,20503.140,20503.140,20503.14,20503.140,20503.140,20503.140,20503.140,14.0,20503.14,19059.702143,295.01,5400.849921,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,6,5,3,4,20503.14
13349736,19746.0,854.077007,6619.089476,1.88,18.04,18.64,19

In [43]:
train_copy = df_training.copy()

In [44]:
# Allow only maximum of 2 NaN per row or remove the rows with more than 2 NaN values Eg: band_std = NaN and promoter_std = NaN
train_copy.dropna(axis=0, thresh=152, inplace=True)
display(train_copy.head(5))
print(train_copy.shape)
#(195254, 154)

,vg_state_count,vg_state_mean,vg_state_std,vg_state_min,vg_state_5%,vg_state_10%,vg_state_15%,vg_state_20%,vg_state_25%,vg_state_30%,vg_state_35%,vg_state_40%,vg_state_45%,vg_state_50%,vg_state_55%,vg_state_60%,vg_state_65%,vg_state_70%,vg_state_75%,vg_state_80%,vg_state_85%,vg_state_90%,vg_state_95%,vg_state_max,band_count,band_mean,band_std,band_min,band_5%,band_10%,band_15%,band_20%,band_25%,band_30%,band_35%,band_40%,band_45%,band_50%,band_55%,band_60%,band_65%,band_70%,band_75%,band_80%,band_85%,band_90%,band_95%,band_max,promoter_count,promoter_mean,promoter_std,promoter_min,promoter_5%,promoter_10%,promoter_15%,promoter_20%,promoter_25%,promoter_30%,promoter_35%,promoter_40%,promoter_45%,promoter_50%,promoter_55%,promoter_60%,promoter_65%,promoter_70%,promoter_75%,promoter_80%,promoter_85%,promoter_90%,promoter_95%,promoter_max,place_kirche,place_hotel,place_cafe,place_theater,place_club,place_halle,place_gaststaette,place_festhalle,place_kulturzentrum,place_festzelt,place_schloss,place_pub,place_stadthalle,place_park,place_gasthof,place_kabarett,place_arena,place_schlachthof,place_wandelhalle,place_turnhalle,place_buergerhaus,place_museum,place_rathaus,place_staatsbad,place_zelt,place_jazz,place_forum,place_gymnasium,place_schule,place_sporthalle,band_kurorchester bad wil,band_musikverein harmonie,band_kasalla,band_cat ballou,band_roncalli royal orch,band_jugendblasorchester,band_kurorchester bad pyr,band_hoehner,band_paveier,band_domstuermer,band_kluengelkoepp,band_alleinunterhalter,band_the gregorian voices,band_brings,band_musica hungarica,band_concerto,band_bad salzuflen orches,band_musikverein stadtkap,band_salonorchester hunga,band_miljoe,band_raeuber,band_kabarett leipziger f,band_marita koellner,band_salon-orchester hung,band_blaeck foeoess,band_schuelerinnen und sc,band_romain vicente,band_staatliche kurkapell,band_musikzug der freiwil,band_funky marys,state_bavaria,state_rhineland-palatinate,state_baden-wuerttemberg,state_north rhine-westphalia,state_thuringia,state_hesse,state_brandenburg,state_schleswig-holstein,state_berlin,state_mecklenburg-western pomerania,state_lower saxony,state_hamburg,state_saarland,state_saxony-anhalt,state_saxony,state_bremen,vg_datum_year,vg_datum_month,vg_datum_day_of_week,vg_datum_season,veranst_segment,vg_inkasso
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10985840,428.0,1030.554252,9541.693508,12.24,18.440,18.640,20.566,23.050,25.3025,36.520,43.0475,51.344,61.170,69.73,89.398,109.920,147.578,189.372,235.8075,303.324,398.880,691.273,2687.6785,183608.57,21.0,19.247619,1.405762e+01,15.59,15.7700,15.770,15.770,15.770,15.7700,15.77,15.7700,15.77,15.770,15.77,15.7700,15.770,15.7700,15.94,15.9400,15.940,18.4400,18.440,18.4400,80.46,2.0,21.49,2.206173,19.93,20.086,20.242,20.398,20.554,20.710,20.866,21.022,21.178,21.334,21.49,21.646,21.802,21.958,22.114,22.270,22.426,22.582,22.738,22.894,23.05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2017,10,2,4,2,15.94
13906939,229.0,11936.264192,91332.658153,15.59,16.218,18.440,18.840,19.770,22.8000,23.150,23.5500,33.368,38.938,48.20,64.560,81.470,96.712,134.004,173.4000,261.672,393.024,844.064,1813.2580,818332.70,3.0,16.490000,0.000000e+00,16.49,16.4900,16.490,16.490,16.490,16.4900,16.49,16.4900,16.49,16.490,16.49,16.4900,16.490,16.4900,16.49,16.4900,16.490,16.4900,16.490,16.4900,16.49,2.0,16.11,0.000000,16.11,16.110,16.110,16.110,16.110,16.110,16.110,16.110,16.110,16.110,16.11,16.110,16.110,16.110,16.110,16.110,16.110,16.110,16.110,16.110,16.11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2019,6,3,3,2,16.49
10522377,4.0,106.580000,65.938998,17.44,30.574,43.708,56.842,69.976,83.1100,96.244,106.2400,109.960,113.680,117.40,12

(195254, 154)


## Set NaN values to 0.0 in 'promoter_std', 'band_std', 'vg_state_std' columns

In [74]:
train_copy['promoter_std'] = np.where((train_copy.promoter_std.isna()),0.0,train_copy.promoter_std)
train_copy['band_std'] = np.where((train_copy.band_std.isna()),0.0,train_copy.band_std)
train_copy['vg_state_std'] = np.where((train_copy.vg_state_std.isna()),0.0,train_copy.vg_state_std)

In [75]:
# Check is there are any NaN values in any of the columns
train_copy.isna().any()

vg_state_count                         False
vg_state_mean                          False
vg_state_std                           False
vg_state_min                           False
vg_state_5%                            False
vg_state_10%                           False
vg_state_15%                           False
vg_state_20%                           False
vg_state_25%                           False
vg_state_30%                           False
vg_state_35%                           False
vg_state_40%                           False
vg_state_45%                           False
vg_state_50%                           False
vg_state_55%                           False
vg_state_60%                           False
vg_state_65%                           False
vg_state_70%                           False
vg_state_75%                           False
vg_state_80%                           False
vg_state_85%                           False
vg_state_90%                           False
vg_state_9

## Export training and testing dataframe, each containing 238 features

In [101]:
# df_training.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_training_features_2016_2020.parquet.gzip',compression='gzip')
df_training.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_training_features_2016_2020_v1.parquet.gzip',compression='gzip')

In [102]:
# df_testing.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_testing_features_2016_2020.parquet.gzip',compression='gzip')
df_testing.to_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_testing_features_2016_2020_v1.parquet.gzip',compression='gzip')